In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import math
import collections

In [2]:
dir_path = "/Users/irenechang/Downloads/ECS260/csvs/"

In [3]:
cmt_hist = pd.read_csv(dir_path+"commits_2019_8.csv")

/var/folders/5l/vhc1pp4552v9_1jprtyqnlj80000gn/T/ipykernel_17689/795063772.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  cmt_hist = pd.read_csv(dir_path+"commits_2019_8.csv")


In [4]:
l_ids = pd.read_csv(dir_path+"lists_2019_8.csv",sep=',',header=0,encoding='latin-1')

In [5]:
g_ids = l_ids[l_ids.status==1].listid.to_list()
r_ids = l_ids[l_ids.status==2].listid.to_list()

In [6]:
g_cmts = {}
r_cmts = {}
empty_list = []
for g_id in g_ids:
    start = dt.datetime.strptime(l_ids[l_ids.listid==g_id].start_date.to_string(index=False),"%m/%d/%Y")
    end = dt.datetime.strptime(l_ids[l_ids.listid==g_id].end_date.to_string(index=False),"%m/%d/%Y")
    hist = sorted([dt.datetime.strptime(c[:10],"%Y-%m-%d") for c in cmt_hist[cmt_hist.list==g_id].commit_datetime])
    hist = [h for h in hist if start <= h <= end]
    if len(hist):
        g_cmts[g_id] = hist
    else:
        empty_list.append(g_id)
for r_id in r_ids:
    start = dt.datetime.strptime(l_ids[l_ids.listid==r_id].start_date.to_string(index=False),"%m/%d/%Y")
    end = dt.datetime.strptime(l_ids[l_ids.listid==r_id].end_date.to_string(index=False),"%m/%d/%Y")
    hist = sorted([dt.datetime.strptime(c[:10],"%Y-%m-%d") for c in cmt_hist[cmt_hist.list==r_id].commit_datetime])
    hist = [h for h in hist if start <= h <= end]
    if len(hist):
        r_cmts[r_id] = hist
    else:
        empty_list.append(r_id)

In [7]:
def avg_cmt(freq):
    cnt = 0
    for f in freq:
        cnt += f[1]
    duration = freq[-1][0]-freq[0][0]
    return cnt/duration.days

def get_inactive_period(l_id,g):
    if l_id in empty_list:
        return [],[]
    a_periods = []
    i_periods = []
    cmts = (g_cmts,r_cmts)
    counter = collections.Counter(cmts[g][l_id])
    freq = [(x,counter[x]) for x in sorted(counter.keys())]
    p1 = 0
    p2 = 1
    prev = 0
    total_per = freq[-1][0]-freq[0][0]
    avg = avg_cmt(freq)
    while p2 < len(freq):
        if freq[p2][1] < 0.2*avg: # threshold 1
            p2 += 1
            continue
        else:
            delta = freq[p2][0]-freq[p1][0]
            if delta.days > 60: # threshold 2
                period = ((freq[prev][0]).strftime("%Y-%m-%d"),(freq[p1][0]).strftime("%Y-%m-%d"))
                a_periods.append(period)
                period = ((freq[p1][0]+dt.timedelta(days=1)).strftime("%Y-%m-%d"),(freq[p2][0]-dt.timedelta(days=1)).strftime("%Y-%m-%d"))
                i_periods.append(period)
                prev = p2
            p1 = p2
            p2+=1
    period = ((freq[prev][0]).strftime("%Y-%m-%d"),(freq[-1][0]).strftime("%Y-%m-%d"))
    a_periods.append(period)
    return a_periods,i_periods

In [8]:
new_list = pd.read_excel(dir_path+"new_asfi_list.xlsx") # I removed the empty rows
cnt = 0
for l_id in new_list.listid:
    i_per = get_inactive_period(l_id, int(new_list.loc[new_list.listid==l_id,"status"].to_string(index=False))-1)[1]
    if len(i_per):
        cnt += 1
        i_per = np.array(i_per)
        new_list.loc[new_list.listid==l_id,'downturn_start'] = ','.join(i_per[:,0])
        new_list.loc[new_list.listid==l_id,'downturn_end'] = ','.join(i_per[:,1])
print(cnt)
new_list

15


,listid,listname,start_date,end_date,status,incubation_months,pj_url,pj_github_url,language,downturn_start,downturn_end
0,59,Apex,2015-08-17,2016-04-20,1,8.233333,http://incubator.apache.org/projects/apex.html,https://github.com/apache/apex-core,Java,NaN,NaN
1,62,AsterixDB,2015-02-28,2016-04-20,1,13.900000,http://incubator.apache.org/projects/asterixdb...,https://github.com/apache/AsterixDB,Java,NaN,NaN
2,63,Atlas,2015-05-05,2017-06-21,1,25.933333,http://incubator.apache.org/projects/atlas.html,https://github.com/apache/Atlas,Java,NaN,NaN
3,65,Beam,2016-02-01,2016-12-21,1,10.800000,http://incubator.apache.org/projects/beam.html,https://github.com/apache/Beam,Java,NaN,NaN
4,70,Brooklyn,2014-05-01,2015-11-18,1,18.866667,http://incubator.apache.org/projects/brooklyn....,https://github.com/apache/incubator-brooklyn,Java,NaN,NaN
5,72,Calcite,2014-05-19,2015-10-22,1,17.366667,http://incubator.apache.org/projects/calcite.html,https://github.com/apache/Calcite,Java,NaN,NaN
6,73,CarbonData,2016-06-02,2017-04-19,1,10.700000,http://incubator.apache.org/projects/carbondat...,https://github.com/apache/CarbonData,Scala,NaN,NaN
7,82,CommonsRDF,2015-03-06,2016-11-28,1,21.100000,http://incubator.apache.org/projects/commonsrd...,https://github.com/apache/commons-rdf,Java,"2015-05-16,2015-12-11","2015-08-09,2016-03-11"
8,89,DataFu,2014-01-05,2018-02-21,1,50.266667,http://incubator.apache.org/projects/datafu.html,https://github.com/apache/DataFu,Java,"2014-11-28,2016-03-10,2016-10-27,2017-03-11","2015-05-26,2016-06-27,2017-02-12,2017-07-11"
9,96,DistributedLog,2016-06-24,2017-07-01,1,12.400000,http://incubator.apache.org/projects/distribut...,https://github.com/apache/DistributedLog,Java,NaN,NaN


In [9]:
new_list.to_csv(dir_path+"new_list_w_downturns.csv",index=False)